### IMPORTS

In [5]:
import ollama
import json
import msvcrt
import whisper
import speech_recognition as sr
import torch
import numpy as np
import wave

### LLM MODEL

In [6]:
LLM_MODEL = "mistral"

### VOICE RECOGNITION 

In [7]:
MODEL_NAME = "turbo" 

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading {MODEL_NAME} model on {device}...")
model = whisper.load_model(MODEL_NAME, device=device)

def audio_by_user():
    recognizer = sr.Recognizer()
    with sr.Microphone(sample_rate=16000) as source: # Force 16kHz
        print("Say: 'Robot, move forward one meter.'")
        
        print("Listening...")
        audio = recognizer.listen(source, timeout=6, phrase_time_limit=10)
        
        with open("debug_audio.wav", "wb") as f:
            f.write(audio.get_wav_data())
        print("Audio saved to 'debug_audio.wav'. Listen to it to check for noise/static.")

        # Transcribe
        audio_data = np.frombuffer(audio.get_raw_data(), np.int16).flatten().astype(np.float32) / 32768.0
        result = model.transcribe(audio_data, language="en", fp16=torch.cuda.is_available())
        
        user_speech = print(f"\nWhisper Result: \"{result['text'].strip()}\"")

    return user_speech


Loading turbo model on cuda...


### LLM PROCESSING FOR ROBOT CODE GENERATION

In [8]:
def get_mission_plan(user_input):
    print(f"[BRAIN] Processing logic with {LLM_MODEL}...")

    # STRICT System Prompt
    # We tell the LLM exactly what objects exist so it doesn't hallucinate "yellow_box"
    system_manual = """
    You are a Robot Logic Unit.
    
    ENVIRONMENT:
    - OBJECTS: ['red_box', 'green_box', 'blue_box']
    - LOCATIONS: ['home_position', 'table_center']
    
    COMMANDS ALLOWED:
    - 'pick_up(target)'
    - 'move_to(target)'
    - 'place_at(target)'
    
    TASK:
    Output a JSON object containing a list of actions based on the user's command.
    
    Example Input: "Grab the red box and put it on the table."
    Example Output: {
        "plan": [
            {"action": "pick_up", "target": "red_box"},
            {"action": "place_at", "target": "table_center"}
        ]
    }
    """

    try:
        response = ollama.generate(
            model=LLM_MODEL,
            system=system_manual,
            prompt=f"User Command: {user_input}",
            format="json",  # Forces JSON output mode in Ollama
            options={"temperature": 0} # 0 makes it deterministic (logic > creativity)
        )
        
        raw_response = response['response']
        print(f"[BRAIN] Raw Model Output: {raw_response}") # Debug print
        
        # Parse JSON
        data = json.loads(raw_response)
        
        # Normalize: Try to find a list of actions
        if "plan" in data:
            return data["plan"]
        elif isinstance(data, list):
            return data
        else:
            # If it returns a single object, wrap it in a list
            return [data]

    except Exception as e:
        print(f"[BRAIN] Error: {e}")
        return []